<a href="https://colab.research.google.com/github/RDGopal/IB9CW0-Text-Analytics/blob/main/day_eight_simple_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple RAG System using Llama Index
In this tutorial we will build a full RAG pipeline using Llama Index. We will install an open-source, smallish LLM [Phi-3](https://news.microsoft.com/source/features/ai/the-phi-3-small-language-models-with-big-potential/), from Microsoft, to run locally on our Colab instance to do the generation. To augment the results we will give the LLM access to our ArXiv abstracts this time stored in [Chroma db](https://www.trychroma.com/) as a vector store/document store. Finally, we use [Ollama](https://ollama.com/) as an interface to interact with Phi-3 on our machine.

We'll start by installing Ollama:

In [ ]:
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10091    0 10091    0     0  31158      0 --:--:-- --:--:-- --:--:-- 31145
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Next we do a bit of work to get Ollama running in the background of our Colab (Linux) instance. Don't worry too much about this code as it is mostly Linux/BASH commands.

In [ ]:
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # print the global variable to check it saved

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

phi:latest
Process ID: 525
ollama version is 0.1.33


Now that everything should be set up, we can query the model and get it to generate some text about WBS:

In [ ]:
# Query the model via the command line
# First time running it will "pull" (import) the model
!ollama run $OLLAMA_MODEL "Give me short summary of Warwick Business School."

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 047

With everything seemingly working we can start to build our RAG. Firstly, this involves installing a bunch of packages:

In [ ]:
# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)


With everything installed we can setup (in Python) our LLM model:

In [ ]:
# Use the global variable (OLLAMA_MODEL) as our LLM
# Set a timeout of 4 minutes
llm = Ollama(model=OLLAMA_MODEL, request_timeout=240.0)

We have a generator but now we needed a retriever to augment our results. As before, we will extract 500 abstracts from ArXiv as our documents> This is basically the same code as the previous Notebook, except that, rather than building a Pandas DataFrame, instead we just store each abstract as a seperate document:

In [ ]:
!mkdir -p '/content/data/' # create an empty directory called "data"
!pip install arxiv

import arxiv

# Number of records
n_records = 500

# Construct the default API client.
client = arxiv.Client()

# Search for the n most recent articles matching the keyword "text analytics."
search = arxiv.Search(
  query = "text analytics", # search query
  max_results = n_records, # number of records to return - defined above
  sort_by = arxiv.SortCriterion.SubmittedDate # sort order (submission date)
)

results = client.results(search) # collect results

count = 0

for r in client.results(search): # iterate through the results
  doc = r.summary
  # create a file name as a combination of the path ("/content/data/") ...
  # the name "Ouput", a number that increases with each abstract (0-499) ...
  # and the file type (".txt").
  # e.g. the first file will be "/content/data/Output0.txt" ...
  # and the last will be "/content/data/Output499.txt"
  fname = "/content/data/Output" + str(count) + ".txt"
  with open(fname, "w") as text_file:
    text_file.write(doc) # save the file
  count += 1 # increment the count

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=e49f5427c953beb0a4e04e55f9dead12c4a35d3c1afbea5ad084e95afebda30c
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


With all our documents created, we can check the first one to make sure everything worked:

In [ ]:
first_file = open('/content/data/Output0.txt', 'r')
file_contents = first_file.read() # read the document
print(file_contents)
first_file.close() # close when finished

Recent advancements in Large Language Models (LLMs) have led to the
development of Video Large Multi-modal Models (Video-LMMs) that can handle a
wide range of video understanding tasks. These models have the potential to be
deployed in real-world applications such as robotics, AI assistants, medical
imaging, and autonomous vehicles. The widespread adoption of Video-LMMs in our
daily lives underscores the importance of ensuring and evaluating their robust
performance in mirroring human-like reasoning and interaction capabilities in
complex, real-world contexts. However, existing benchmarks for Video-LMMs
primarily focus on general video comprehension abilities and neglect assessing
their reasoning capabilities over complex videos in the real-world context, and
robustness of these models through the lens of user prompts as text queries. In
this paper, we present the Complex Video Reasoning and Robustness Evaluation
Suite (CVRR-ES), a novel benchmark that comprehensively assesses the
perf

Now we will use Llama Index's [SimpleDirectoryReader](https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/) to load all the documents. After this we will specify an embedding model from HuggingFace and add all of this to Llama Index's settings:

In [ ]:
# Load documents
reader = SimpleDirectoryReader("/content/data") # load documents from the /data folder
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 500 docs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Now we will setup a vector database (this time using Chroma DB) and index our documents (via embeddings). As in the previous tutorial, this will allow us to search for relevant documents to our query:

In [ ]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")

# Create a collection/table ("demo-for-ram") in the db
chroma_collection = db.create_collection("demo-for-ram")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs, # the abstract documents (e.g. "Output0.txt")
    storage_context = storage_context,
    embed_model = embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

name='demo-for-ram' id=UUID('2ebe4afb-923d-4b8f-8112-d5d5cf144add') metadata=None tenant='default_tenant' database='default_database'
Collection name is: demo-for-ram


Now everything is set up we can start (retrieval augmented) generating! Let's try a simple query about LLMs:

In [ ]:
query_engine = index.as_query_engine() # use the vector db for queries

response = query_engine.query("What are LLMs?") # query Phi-3 with context
response.response # print the response

' LLMs (large language models) are advanced machine learning models that have achieved remarkable performance in various applications. They consist of multiple layers of artificial neural networks that work together to generate human-like language processing capabilities. Some popular examples include GPT-3 and GPT-4, which have been used for a variety of tasks such as natural language understanding, generation, and translation. LLMs use deep learning techniques to analyze large datasets and learn the underlying patterns in language, making them more effective than traditional rule-based systems.\n'

And there we have it! A working RAG running in our notebook. To understand a little better the processes behind generating this output, we can inspect the output metadata. Alongside the above response, we can also see the documents retreived to augment the generation. Feel free to look up those documents (code cell 7) to see how they may have influenced the result.

In [ ]:
response.metadata # print the full output from the LLM

{'5007abfd-efad-4238-84c7-ba2d93061574': {'file_path': '/content/data/Output202.txt',
  'file_name': 'Output202.txt',
  'file_type': 'text/plain',
  'file_size': 1574,
  'creation_date': '2024-05-07',
  'last_modified_date': '2024-05-07'},
 '4d26ba57-748a-452a-83ae-4ba49b59804f': {'file_path': '/content/data/Output35.txt',
  'file_name': 'Output35.txt',
  'file_type': 'text/plain',
  'file_size': 1462,
  'creation_date': '2024-05-07',
  'last_modified_date': '2024-05-07'}}

And that's it! Well done 👍